<a href="https://colab.research.google.com/github/secant78/Naive-Bayes-and-Logistic-Regression/blob/main/CISC_684_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#TODO: implement perceptron
#MCAT Logistic algorithm
#Naive Bayes - done
#import training and test set - done

import os
import math
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

#"/content/drive/My Drive/"
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import data set to list
import glob
import pandas as pd

# get data file names
pathHam =r'/content/drive/My Drive/CISC 684/dataset 1/train/ham'
pathSpam = r'/content/drive/My Drive/CISC 684/dataset 1/train/spam'

pathHamData2 = r'/content/drive/My Drive/CISC 684/dataset 2/train/ham'
pathSpamData2 = r'/content/drive/My Drive/CISC 684/dataset 2/train/spam'


pathHamData3 = r'/content/drive/My Drive/CISC 684/dataset 3/train/ham'
pathSpamData3 = r'/content/drive/My Drive/CISC 684/dataset 3/train/spam'


pathTestHam = r'/content/drive/My Drive/CISC 684/dataset 1/test/ham'
pathTestSpam = r'/content/drive/My Drive/CISC 684/dataset 1/test/spam'


pathTestHamData2 = r'/content/drive/My Drive/CISC 684/dataset 2/test/ham'
pathTestSpamData2 = r'/content/drive/My Drive/CISC 684/dataset 2/test/spam'


pathTestHamData3 = r'/content/drive/My Drive/CISC 684/dataset 3/test/ham'
pathTestSpamData3 = r'/content/drive/My Drive/CISC 684/dataset 3/test/spam'


filenamesHam = glob.glob(pathHam + "/*.txt")
filenamesSpam = glob.glob(pathSpam + "/*.txt")


filenamesHamData2 = glob.glob(pathHamData2 + "/*.txt")
filenamesSpamData2 = glob.glob(pathSpamData2 + "/*.txt")


filenamesHamData3 = glob.glob(pathHamData3 + "/*.txt")
filenamesSpamData3 = glob.glob(pathSpamData3 + "/*.txt")


filenameTestHam = glob.glob(pathTestHam + "/*.txt")
filenameTestSpam = glob.glob(pathTestSpam + "/*.txt")


filenameTestHamData2 = glob.glob(pathTestHamData2 + "/*.txt")
filenameTestSpamData2 = glob.glob(pathTestSpamData2 + "/*.txt")


filenameTestHamData3 = glob.glob(pathTestHamData2 + "/*.txt")
filenameTestSpamData3 = glob.glob(pathTestSpamData2 + "/*.txt")

dfsHam = []
dfsSpam = []


dfsHamData2 = []
dfsSpamData2 = []

dfsHamData3 = []
dfsSpamData3 = []

dfsTestHam = []
dfsTestSpam = []

dfsTestHamData2 = []
dfsTestSpamData2 = []

dfsTestHamData3 = []
dfsTestSpamData3 = []


In [ ]:
#adds all file lines into an array
def readFile(fileName):
        fileObj = open(fileName, "r", encoding="ISO-8859-1") #opens the file in read mode
        words = fileObj.read() #puts the file into an array
        #words['a'] = words['a'].str.join(" ") 
        #print(words)
        #words = unicode(words, errors='ignore')
        fileObj.close()
        words = words.translate(str.maketrans('', '', string.punctuation))    # Remove punctuation
        tokens = nltk.word_tokenize(words)

        #stopWords = set(stopwords.words('english'))         # Get list of stop words
        terms = []
        for word in tokens:
          if word.isalpha():
            word = word.lower()                         # Convert word to lower case
            word = wn.morphy(word)                      # Convert word to its stem
            terms.append(word)                          # Add stemmed word to words
        return terms

       # return words

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def tokenizeFiles(dfs, filenames):
  for filename in filenames:
    dfs.append(readFile(filename))  
  return dfs


In [ ]:
dfsHam = tokenizeFiles(dfsHam,filenamesHam)

In [ ]:
dfsSpam = tokenizeFiles(dfsSpam,filenamesSpam)

In [ ]:
dfsHamData2 = tokenizeFiles(dfsHamData2,filenamesHamData2)

In [ ]:
dfsSpamData2 = tokenizeFiles(dfsSpamData2,filenamesSpamData2)

In [ ]:
dfsHamData3 = tokenizeFiles(dfsHamData3,filenamesHamData3)

In [ ]:
dfsSpamData3 = tokenizeFiles(dfsSpamData3,filenamesSpamData3)

In [ ]:
dfsTestHam = tokenizeFiles(dfsTestHam,filenameTestHam)

In [ ]:
dfsTestSpam = tokenizeFiles(dfsTestSpam, filenameTestSpam)

In [ ]:
dfsTestHamData2 = tokenizeFiles(dfsTestHamData2,filenameTestHamData2)

In [ ]:
dfsTestSpamData2 = tokenizeFiles(dfsTestSpamData2, filenameTestSpamData2)

In [ ]:
dfsTestHamData3 = tokenizeFiles(dfsTestHamData3,filenameTestHamData3)

In [ ]:
dfsTestSpamData3 = tokenizeFiles(dfsTestSpamData3, filenameTestSpamData3)

MultiNomial Naive Bayes

In [ ]:
import numpy as np
import math


#makes a dictionary of word counts in file
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))



#takes lists of all words in training set and passes them to wordListToFreqDict()
#which converts the words into word frequencies. Then it combines the word frequencies
#of each file into one dictionary called setTerms which has all unique keys with word counts
#of all files

def getWords(dfs):

  terms = {}
  setTerms = {}
  count = 0

  for file in dfs:
    terms[str(count)] = wordListToFreqDict(file)
    count +=1
  terms = dict((k, v) for k, v in terms.items() if k is not None)
  for filewords in terms:
    for word in terms[filewords]:
      if word in setTerms:
        setTerms[word]+= terms.get(filewords, {}).get(word)
      else:
        setTerms[word] = terms.get(filewords, {}).get(word)
  final_dict = dict((k, v) for k, v in setTerms.items() if k is not None)
  return final_dict

def getWords2(array):
  terms = {}
  setTerms = {}
  terms['0'] = wordListToFreqDict(array)
  for filewords in terms:
    for word in terms[filewords]:
      if word in setTerms:
        setTerms[word]+= terms.get(filewords, {}).get(word)
      else:
        setTerms[word] = terms.get(filewords, {}).get(word)
  return setTerms


#data strutures for doing the analysis / algorithms
termsHam = getWords(dfsHam)
#print(termsHam)
termsSpam = getWords(dfsSpam)

termsHamData2 = getWords(dfsHamData2)
termsSpamData2 = getWords(dfsSpamData2)

termsHamData3 = getWords(dfsHamData3)
termsSpamData3 = getWords(dfsSpamData3)

termsTestHam = getWords(dfsTestHam)
termsTestSpam = getWords(dfsTestSpam)
#print(termsHam)
#print(termsSpam)



#calculate fraction of ham and spam in entire data set and their probabilities
nHam = len(dfsHam)          # Find the number of ham files there are in the training data
nSpam = len(dfsSpam)        # Find the number of spam files there are in the training data
pHam = nHam / (nHam + nSpam)        # Find the probability that a file is of class 'ham'
pSpam = nSpam / (nHam + nSpam)


def initial_prob(dfsHam,dfsSpam):
  nHam = len(dfsHam)         
  nSpam = len(dfsSpam)        
  pHam = nHam / (nHam + nSpam)        
  pSpam = nSpam / (nHam + nSpam)

  return pHam

probHamData2 = initial_prob(dfsHamData2,dfsSpamData2)
probSpamData2 = 1-probHamData2
probHamData2 = math.log10(probHamData2)
probSpamData2 = math.log10(probSpamData2)


probHamData3 = initial_prob(dfsHamData3,dfsSpamData3)
probSpamData3 = 1-probHamData3
probHamData3 = math.log10(probHamData3)
probSpamData3 = math.log10(probSpamData3)


probabilityHam = math.log10(pHam)
probabilitySpam = math.log10(pSpam)



#calculates the denominator of term probability formula. passed onto termProbability function
def getTermDenominator(termdict):
  wordCount = sum(termdict.values())
  m = len(termdict)
  return wordCount + m

#calculates term probability based on term and term dictionary and getTermDenominator function
def termProbability(term,termdict):
  denominator = getTermDenominator(termdict)
  termCount = 0
  if term in termdict:
    termCount += termdict[term]
  else:
    termCount = 0 
  probability = (termCount + 1)/(denominator)
  return probability




def probabilityClassification(test_data, probClass, termDictionary):
  

  termprobabilities = []
  for term in test_data:
        termprobabilities.append(math.log10(termProbability(term, termDictionary)))
  #print(termprobabilities)
  total = 0
  for elements in range(0, len(termprobabilities)): 
    total += termprobabilities[elements] 
  total_probability = probClass + total
  #print(probClass)
  #print(total)
  return total_probability

hamProbLog = probabilityClassification(termsTestHam, probabilityHam, termsHam)
spamProbLog = probabilityClassification(termsTestSpam, probabilitySpam, termsSpam)

print(hamProbLog, spamProbLog)

def finalClassification(ham, spam):
  if spam > ham:
    return 1
  else:
    return 0

classification = finalClassification(hamProbLog,spamProbLog)

print(classification)

-11293.413990933324 -18965.072892477325
0


In [ ]:
#accuracy algorithm

nCorrect = 0
nIncorrect = 0
    
for file in dfsTestHam:
  testHamWords = getWords2(file)
  log_prob_file_ham = probabilityClassification(testHamWords, probabilityHam, termsHam)
  log_prob_file_spam = probabilityClassification(testHamWords,probabilitySpam, termsSpam)
  final_class = finalClassification(log_prob_file_ham,log_prob_file_spam)
  #print(final_class)
  if final_class == 0:
    nCorrect += 1
  else:
    nIncorrect += 1

for file in dfsTestSpam:
  testSpamWords = getWords2(file)
  log_prob_file_ham1 = probabilityClassification(testSpamWords, probabilityHam, termsHam)
  log_prob_file_spam1 = probabilityClassification(testSpamWords,probabilitySpam, termsSpam)
  final_class1 = finalClassification(log_prob_file_ham1,log_prob_file_spam1)
  #print(final_class1)
  if final_class1 == 1:
    nCorrect += 1
  else:
    nIncorrect += 1



final_accuracy = nCorrect / (nCorrect + nIncorrect) * 100


print('Final Accuracy for Data Set 1:')
print(final_accuracy)


def accuracy(listTestHam, listTestSpam, probabHam, probabSpam, termsHam_, termsSpam_):
  numCorrect = 0
  numIncorrect = 0
    
  for file in listTestHam:
    testHamWords = getWords2(file)
    log_prob_file_ham = probabilityClassification(testHamWords, probabHam, termsHam_)
    log_prob_file_spam = probabilityClassification(testHamWords,probabSpam, termsSpam_)
    final_class = finalClassification(log_prob_file_ham,log_prob_file_spam)
    #print(final_class)
    if final_class == 0:
      numCorrect += 1
    else:
      numIncorrect += 1

  for file in listTestSpam:
    testSpamWords = getWords2(file)
    log_prob_file_ham1 = probabilityClassification(testSpamWords, probabHam, termsHam_)
    log_prob_file_spam1 = probabilityClassification(testSpamWords,probabSpam, termsSpam_)
    final_class1 = finalClassification(log_prob_file_ham1,log_prob_file_spam1)
    #print(final_class1)
    if final_class1 == 1:
      numCorrect += 1
    else:
      numIncorrect += 1



  final_acc = numCorrect / (numCorrect + numIncorrect) * 100
  return final_acc

accuracy_data_2 = accuracy(dfsTestHamData2,dfsTestSpamData2,probHamData2,probSpamData2,termsHamData2,termsSpamData2)
accuracy_data_3 = accuracy(dfsTestHamData3,dfsTestSpamData3,probHamData3,probSpamData3,termsHamData3,termsSpamData3)
  
print('Final Accuracy for Data Set 2:')
print(accuracy_data_2)

print('Final Accuracy for Data Set 3:')
print(accuracy_data_3)


Final Accuracy for Data Set 1:
94.35146443514645
Final Accuracy for Data Set 2:
94.51754385964912
Final Accuracy for Data Set 3:
81.57894736842105


Logistic Regression

In [ ]:

#variables not yet defined: weight, data, partial_deriv_loss, lam



#formula for when you wanna classify spam (P=1)
def log_formula(h):
  #print(1/(1+np.exp(-1*h)))
  return 1/(1+np.exp(-1*h))

def log_formula_zero(h):
  return np.exp(-h)/(1+np.exp(-h))

def h_function(weights, x_matrix):
  return weights[0] + weights[1]*x_matrix.iloc[:,1] + weights[2]*x_matrix.iloc[:,2]

#formula for loss function
#yPredY_One = output of log_formula
#yPredY_Zero = output of log_formula_zero
#y = pHam
#one_minus_y = pSpam
def loss_function(weights,x_matrix,probabHam):
  y_predicted = h_function(weights,x_matrix)
  check = probabHam * np.log10(y_predicted) + (1-probabHam) * np.log10(1-y_predicted)
  check = check.replace([-np.inf], -100)
  check = check.replace([np.inf], 100)
  #print(check)
  return -sum(check)

#formula for gradient ascent (still needs to be done)
def gradient(weight, h_funct, probabHam, df_X):
  g = [0,0,0]
  y_pred = log_formula(h_funct)
  g[0] = -1 * sum(probabHam*(y_pred) - (1-probabHam) * (1-y_pred))
  g[1] = -1 * sum(probabHam*(y_pred)*df_X.iloc[:,1] - (1-probabHam)*(1-y_pred)*df_X.iloc[:,1])
  g[2] = -1 * sum(probabHam*(y_pred)*df_X.iloc[:,2] - (1-probabHam)*(1-y_pred)*df_X.iloc[:,2])
  return g

def ascent(w_new,w_prev,lr,x_matrix):
  j=0
  while True:
    w_prev = w_new
    
    w0 = w_prev[0] + lr*gradient(w_prev,h_function(w_prev,x_matrix),pHam,x_matrix)[0]
    w1 = w_prev[1] + lr*gradient(w_prev,h_function(w_prev,x_matrix),pHam,x_matrix)[1]
    w2 = w_prev[2] + lr*gradient(w_prev,h_function(w_prev,x_matrix),pHam,x_matrix)[2]
    w_new = [w0,w1,w2]
    if((w_new[0] - w_prev[0])**2 + (w_new[1] - w_prev[1])**2 + (w_new[2] - w_prev[2])**2 < pow(10,6)):
      return w_new
    if j > 100:
      return w_new
    j+=1

#initialize weights, will be updated with every call of the gradient ascent function
weight0 = 1
weight_ham = 1
weight_spam = 1
weight = [1,1,1]


#stores the words contained in all ham and spam files for training data
all_words = []

#appends words in ham training data
for word in termsHam:
  all_words.append(word)
#appends words in spam training data
for word in termsSpam:
  all_words.append(word)

#part of converting all_words into a data frame
all_words = list(dict.fromkeys(all_words))
#print(all_words)

#converts all_words into data frame
all_words_df = pd.DataFrame(all_words, columns = ['words'])
#print(all_words_df)





#part of combining the ham and spam training file words into a data frame
train_ham_df = pd.DataFrame(list(termsHam.items()), columns = ['words','frequency_x0'])
train_spam_df = pd.DataFrame(list(termsSpam.items()), columns = ['words', 'frequency_x1'])

check_df = all_words_df.merge(train_ham_df, how='left', left_on='words', right_on='words')
check_df = check_df.merge(train_spam_df, how='left', left_on='words', right_on='words')


check_df = check_df.fillna(0)

weight = ascent(weight,weight,.0099,check_df)

print(weight)








[-33.77977573681067, -218.80786890693085, -88.56349250177685]


In [ ]:
def accuracy_log(listTestHam, listTestSpam, probabHam, probabSpam, termsHam_, termsSpam_):
  numCorrect = 0
  numIncorrect = 0
    
  for file in listTestHam:
    testHamWords = getWords2(file)
    word_list = []

    #appends words in ham training data
    for word in testHamWords:
      word_list.append(word)
    #appends words in spam training data
    for word in testHamWords:
      word_list.append(word)

    #part of converting all_words into a data frame
    word_list = list(dict.fromkeys(word_list))
    #print(all_words)

    #converts all_words into data frame
    word_list_df = pd.DataFrame(word_list, columns = ['words'])
    #print(all_words_df)





    #part of combining the ham and spam training file words into a data frame
    test_ham_df = pd.DataFrame(list(word_list_df.items()), columns = ['words','frequency_x0'])
    test_spam_df = pd.DataFrame(list(word_list_df.items()), columns = ['words', 'frequency_x1'])

    test_df = all_words_df.merge(test_ham_df, how='left', left_on='words', right_on='words')
    test_df = check_df.merge(test_spam_df, how='left', left_on='words', right_on='words')

    test_df = test_df.fillna(0)
    
    #def log_formula(h):
    #  print(1/(1+np.exp(-1*h)))
    #  return 1/(1+np.exp(-1*h))

    #def h_function(weights, x_matrix):

    h_var = h_function(weight, test_df)
    file_classification_ham = log_formula(h_var)
    file_classification_spam = log_formula_zero(h_var)

    classification_ham_sum = file_classification_ham.sum()
    classification_spam_sum = file_classification_spam.sum()

    #print(classification_ham_sum)
    #print(classification_spam_sum)
    final_class = finalClassification(classification_ham_sum,classification_spam_sum)
    #print(final_class)
    if final_class == 1:
      numCorrect += 1
    else:
      numIncorrect += 1
  '''
  for file in listTestSpam:
    testSpamWords = getWords2(file)
    log_prob_file_ham1 = probabilityClassification(testSpamWords, probabHam, termsHam_)
    log_prob_file_spam1 = probabilityClassification(testSpamWords,probabSpam, termsSpam_)
    final_class1 = finalClassification(log_prob_file_ham1,log_prob_file_spam1)
    #print(final_class1)
    if final_class1 == 1:
      numCorrect += 1
    else:
      numIncorrect += 1
  '''
  for file in listTestSpam:
    testSpamWords = getWords2(file)
    word_list = []

    #appends words in ham training data
    for word in testSpamWords:
      word_list.append(word)
    #appends words in spam training data
    for word in testSpamWords:
      word_list.append(word)

    #part of converting all_words into a data frame
    word_list = list(dict.fromkeys(word_list))
    #print(all_words)

    #converts all_words into data frame
    word_list_df = pd.DataFrame(word_list, columns = ['words'])
    #print(all_words_df)

    #part of combining the ham and spam training file words into a data frame
    test_ham_df = pd.DataFrame(list(word_list_df.items()), columns = ['words','frequency_x0'])
    test_spam_df = pd.DataFrame(list(word_list_df.items()), columns = ['words', 'frequency_x1'])

    test_df = all_words_df.merge(test_ham_df, how='left', left_on='words', right_on='words')
    test_df = check_df.merge(test_spam_df, how='left', left_on='words', right_on='words')

    test_df = test_df.fillna(0)
    
    #def log_formula(h):
    #  print(1/(1+np.exp(-1*h)))
    #  return 1/(1+np.exp(-1*h))

    #def h_function(weights, x_matrix):

    h_var = h_function(weight, test_df)
    file_classification_ham = log_formula(h_var)
    file_classification_spam = log_formula_zero(h_var)

    classification_ham_sum = file_classification_ham.sum()
    classification_spam_sum = file_classification_spam.sum()

    #print(classification_ham_sum)
    #print(classification_spam_sum)
    final_class = finalClassification(classification_ham_sum,classification_spam_sum)
    #print(final_class)
    if final_class == 0:
      numCorrect += 1
    else:
      numIncorrect += 1


  final_acc = numCorrect / (numCorrect + numIncorrect) * 100
  return final_acc

accuracy_data_log = accuracy_log(dfsTestHam,dfsTestSpam,pHam,pSpam,termsHam,termsSpam)
print("Accuracy for Data Set 1:")
print(accuracy_data_log)
accuracy_data_2_log = accuracy_log(dfsTestHamData2,dfsTestSpamData2,probHamData2,probSpamData2,termsHamData2,termsSpamData2)
print("Accuracy for Data Set 2:")
print(accuracy_data_2_log)
accuracy_data_3_log = accuracy_log(dfsTestHamData3,dfsTestSpamData3,probHamData3,probSpamData3,termsHamData3,termsSpamData3)
print("Accuracy for Data Set 3:")
print(accuracy_data_3_log)


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


Accuracy for Data Set 1:
72.80334728033473
Accuracy for Data Set 2:
67.32456140350878
Accuracy for Data Set 3:
67.32456140350878
